# Objetivo

El caso:
Se trata de un conjunto de supermercados que tienen varios años de datos acumulados de productos de limpieza del hogar.


Realizan promociones de forma permanente y quieren poder predecir el volumen de ventas según las promociones que realizan.

Objetivo:
Se trata de plantear como abordar el problema completo, desde que el cliente plantea la necesidad:

Como abordar el problema?
Fases del proyecto y del proceso analítico?
Tecnología a utilizar según las fases?
Arquitectura según las fases?
Propuestas de valor para el cliente a lo largo del proyecto?
Esfuerzos necesarios y perfiles necesarios?
Baseline de resultados iniciales?

Trabajo a Realizar:

Se trata de un trabajo de un par de días de dedicación total, con un informe de proyecto de unas 20-30 páginas máximo.

No se trata de realizar o ejecutar todo el proyecto.
Se trata de documentar como se realizaría o ejecutaría, que hace falta para hacerlo y solamente realizar una primera estimación de resultados base con el mínimo esfuerzo posible.


In [53]:
import pandas as pd

# Importar datos 

Importar el dataset para los movimientos con fecha de Julio 16 de 2019. 

In [71]:
movimientos_df = pd.read_csv("./data/VMI_Movimientos-20190716.txt",
                             encoding="utf-8",
                             header=None,
                             delimiter='\t')

In [72]:
movimientos_col_names = [
    'idb', 'id_producto', 'fecha', 'stock_uni', 'venta_uni', 'precio_uni',
    'cd_abast', 'is_prod_rol_local'
]
movimientos_df.columns = movimientos_col_names

In [73]:
movimientos_df.head()

,idb,id_producto,fecha,stock_uni,venta_uni,precio_uni,cd_abast,is_prod_rol_local
0,242,228217,20160715,24.0,0.0,13.63,0,SI
1,242,228217,20160716,24.0,0.0,13.63,0,SI
2,242,228217,20160717,24.0,0.0,13.63,0,SI
3,242,228217,20160718,24.0,0.0,13.63,0,SI
4,242,228217,20160719,24.0,0.0,13.63,0,SI


In [74]:
promo_df = pd.read_csv("./data/VMI_Promociones-20190715.txt",
                       encoding="utf-8",
                       header=None,
                       delimiter='\t')

In [75]:
promo_col_names = [
    'idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'ventas_est_val',
    'nro_prom', 'fecha', 'porc_dcto'
]
promo_df.columns = promo_col_names

In [76]:
promo_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_est_val,nro_prom,fecha,porc_dcto
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


Importar el dataset de los proveedores y productos. 

In [137]:
prov_prod_df = pd.read_csv("./data/VMI_ProvProdu-20190712.txt",
                           encoding="ISO-8859-1",
                           header=None,
                           delimiter='\t')

In [138]:
prov_prod_df = prov_prod_df.iloc[:,1:-1]

In [139]:
prov_prod_col_names = [
    'id_proveedor', 'razon_social', 'id_sector', 'desc_sector', 'id_seccion',
    'desc_seccion', 'id_gran_flia', 'desc_gran_flia', 'id_flia', 'desc_flia',
    'id_sub_flia', 'desc_sub_flia', 'id_articulo', 'id_producto',
    'desc_articulo', 'id_region'
]
prov_prod_df.columns = prov_prod_col_names

In [141]:
prov_prod_df['desc_gran_flia'] = prov_prod_df['desc_gran_flia'].replace(to_replace=r'Ba¤o', value='Baño', regex=True)
prov_prod_df['desc_sector'] = prov_prod_df['desc_sector'].replace(to_replace=r'Almacn', value='Almacén', regex=True)

In [142]:
prov_prod_df.fillna('', inplace=True)
prov_prod_df['desc_articulo'] = prov_prod_df['desc_articulo'].str.lower()

In [143]:
prov_prod_df.head()

,id_proveedor,razon_social,id_sector,desc_sector,id_seccion,desc_seccion,id_gran_flia,desc_gran_flia,id_flia,desc_flia,id_sub_flia,desc_sub_flia,id_articulo,id_producto,desc_articulo,id_region
0,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27003,273920,refill citrico oil escence air wick citricos ...,1
1,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27004,273921,freshmatic airwick lavanda repuesto 250.00 mlt,1
2,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27005,284012,air wick citrico full rep.gratis air wick air ...,1
3,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27012,359572,magnolia y cherry air wick repuesto 175.00 gr,1
4,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,1,Desodorantes en aerosol,27007,415268,freshmatic bosque mµgico aparato air wick lush...,1


In [130]:
locales_df = pd.read_csv("./data/VMI_Locales-20190712.txt",
                         encoding="ISO-8859-1",
                         header=None,
                         delimiter='\t')

In [131]:
locales_col_names = [
    'id_cadena', 'desc_cadena', 'id_region', 'desc_region', 'idb',
    'desc_local', 'latitud', 'longitud', 'pd'
]
locales_df.columns = locales_col_names

In [132]:
locales_df['desc_local'] = locales_df['desc_local'].replace(to_replace=r'Rinc¢n', value='Rincón', regex=True)

In [133]:
locales_df.head()

,id_cadena,desc_cadena,id_region,desc_region,idb,desc_local,latitud,longitud,pd
0,1,Plaza Vea,1,Buenos Aires,9693,9693 - P Vea Villa Urquiza.,NaN,NaN,N
1,1,Plaza Vea,1,Buenos Aires,9694,9694 - P Vea JB Justo.,NaN,NaN,N
2,1,Plaza Vea,1,Buenos Aires,9695,695 - P Vea Acoyte.,0.0,0.0,N
3,1,Plaza Vea,1,Buenos Aires,9699,699 - P Vea Rivadavia.,0.0,0.0,N
4,1,Plaza Vea,1,Buenos Aires,9889,889 - Plaza Vea Carpa de liqui,NaN,NaN,N


# Análisis Descriptivo

In [159]:
merged_df = pd.merge(prov_prod_df, locales_df, on='id_region')
merged_df.head()

,id_proveedor,razon_social,id_sector,desc_sector,id_seccion,desc_seccion,id_gran_flia,desc_gran_flia,id_flia,desc_flia,...,desc_articulo,id_region,id_cadena,desc_cadena,desc_region,idb,desc_local,latitud,longitud,pd
0,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,...,refill citrico oil escence air wick citricos ...,1,1,Plaza Vea,Buenos Aires,9693,9693 - P Vea Villa Urquiza.,NaN,NaN,N
1,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,...,refill citrico oil escence air wick citricos ...,1,1,Plaza Vea,Buenos Aires,9694,9694 - P Vea JB Justo.,NaN,NaN,N
2,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,...,refill citrico oil escence air wick citricos ...,1,1,Plaza Vea,Buenos Aires,9695,695 - P Vea Acoyte.,0.0,0.0,N
3,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,...,refill citrico oil escence air wick citricos ...,1,1,Plaza Vea,Buenos Aires,9699,699 - P Vea Rivadavia.,0.0,0.0,N
4,252,Reckitt Benckiser Argentina S.A.,1,Almacén,4,Limpieza,3,Baño y Hogar,4,Desodorantes de ambientes,...,refill citrico oil escence air wick citricos ...,1,1,Plaza Vea,Buenos Aires,9889,889 - Plaza Vea Carpa de liqui,NaN,NaN,N


In [160]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14350 entries, 0 to 14349
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_proveedor    14350 non-null  int64  
 1   razon_social    14350 non-null  object 
 2   id_sector       14350 non-null  int64  
 3   desc_sector     14350 non-null  object 
 4   id_seccion      14350 non-null  int64  
 5   desc_seccion    14350 non-null  object 
 6   id_gran_flia    14350 non-null  int64  
 7   desc_gran_flia  14350 non-null  object 
 8   id_flia         14350 non-null  int64  
 9   desc_flia       14350 non-null  object 
 10  id_sub_flia     14350 non-null  int64  
 11  desc_sub_flia   14350 non-null  object 
 12  id_articulo     14350 non-null  int64  
 13  id_producto     14350 non-null  int64  
 14  desc_articulo   14350 non-null  object 
 15  id_region       14350 non-null  int64  
 16  id_cadena       14350 non-null  int64  
 17  desc_cadena     14350 non-null 

In [ ]:
merged_df = pd.merge(merged_df, promo_df, on="idb")

In [147]:
merged_df.describe()

,id_proveedor,id_sector,id_seccion,id_gran_flia,id_flia,id_sub_flia,id_articulo,id_producto_x,id_region,id_cadena,idb,latitud,longitud,id_producto_y,fecha_desde,fecha_hasta,ventas_est_val,nro_prom,fecha,porc_dcto
count,2.339075e+07,23390746.0,23390746.0,23390746.0,23390746.0,23390746.0,2.339075e+07,2.339075e+07,23390746.0,2.339075e+07,2.339075e+07,2.057856e+07,2.057856e+07,2.339075e+07,2.339075e+07,2.339075e+07,2.339075e+07,2.339075e+07,2.339075e+07,2.339075e+07
mean,6.309390e+02,1.0,4.0,3.0,4.0,1.0,9.979622e+03,3.899406e+05,1.0,4.585547e+00,7.065579e+03,-3.458282e+01,-5.794420e+01,3.883934e+05,2.017279e+07,2.017281e+07,2.462270e+01,4.712750e+05,2.017272e+07,8.394103e+00
std,5.444794e+02,0.0,0.0,0.0,0.0,0.0,1.300619e+04,9.324810e+04,0.0,2.661784e+00,3.169249e+03,4.395837e+00,6.619960e+00,8.390628e+04,1.079948e+04,1.080525e+04,3.031461e+01,3.759687e+04,1.080004e+04,2.893565e+00
min,2.520000e+02,1.0,4.0,3.0,4.0,1.0,1.088000e+03,1.641490e+05,1.0,2.000000e+00,2.420000e+02,-4.587265e+01,-6.749604e+01,1.641460e+05,2.016072e+07,2.016073e+07,0.000000e+00,4.251960e+05,2.016071e+07,3.000000e+00
25%,3.140000e+02,1.0,4.0,3.0,4.0,1.0,4.044000e+03,3.426790e+05,1.0,2.000000e+00,5.207000e+03,-3.476409e+01,-5.857491e+01,3.426820e+05,2.016102e+07,2.016103e+07,6.000000e+00,4.368300e+05,2.016102e+07,7.000000e+00
50%,3.140000e+02,1.0,4.0,3.0,4.0,1.0,6.551000e+03,4.274445e+05,1.0,4.000000e+00,9.026000e+03,-3.461339e+01,-5.843481e+01,4.274450e+05,2.017042e+07,2.017042e+07,1.252000e+01,4.568070e+05,2.017042e+07,7.000000e+00
75%,1.550000e+03,1.0,4.0,3.0,4.0,1.0,1.102200e+04,4.660690e+05,1.0,4.000000e+00,9.159000e+03,-3.457286e+01,-5.838981e+01,4.583950e+05,2.018072e+07,2.018080e+07,3.044000e+01,5.027020e+05,2.018072e+07,8.000000e+00
max,1.550000e+03,1.0,4.0,3.0,4.0,1.0,7.702300e+04,5.113260e+05,1.0,9.000000e+00,9.977000e+03,0.000000e+00,0.000000e+00,5.075280e+05,2.019071e+07,2.019072e+07,3.008200e+02,5.410660e+05,2.019071e+07,1.800000e+01


In [158]:
print(merged_df.loc[:, ['id_producto_x', 'id_producto_y']])

          id_producto_x  id_producto_y
0                273920         164149
1                273920         164149
2                273920         164149
3                273920         164149
4                273920         164149
...                 ...            ...
23390741         507832         484434
23390742         507832         484434
23390743         507832         484434
23390744         507832         484434
23390745         507832         484434

[23390746 rows x 2 columns]
